# Neural Networks II - Exercises

## IMDb DataSet - Binary Classification

In dieser Übung betrachten wir das IMDb (Internet Movie Database) Dataset. Dieses enthält
50.000 Movie Reviews, die hälftig postiv (y=1) bzw. negativ (y=0) ausfallen (siehe auch https://keras.io/datasets/ $\rightarrow$ IMDB Movie reviews sentiment classification).

- Lesen Sie die Daten ein, wobei Sie nur 1000 meist verwendeten Worte für jeden Review verwenden (option num_words). Die Daten sind in diesem Fall Python-Listen.

In [ ]:
from keras.datasets import imdb
import numpy as np

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words=1000)

# restore np.load for future normal usage
np.load = np_load_old

- Lesen Sie die Dictionary word_index ein, welches  
$$
\mathrm{word}\quad \rightarrow \quad \mathrm{integer\,index}
$$
zuordnet.

In [ ]:
word_index = imdb.get_word_index()

- Erstellen Sie daraus ein Dictionary reverse_word_index, welches 
$$
\mathrm{integer\,index}\quad \rightarrow \quad \mathrm{word}
$$
zuordnet.

In [ ]:
reverse_word_index = dict([(val,key) for (key,val) in word_index.items() ])

- Lesen Sie damit zur Illustration einen beliebigen Review aus.

In [ ]:
reverse_word_index.get(100,'')

In [ ]:
# Read a movie review
' '.join([reverse_word_index.get(i,'') for i in train_data[100]])

- Führen Sie ein One-Hot-Encoding der Training- und Testdaten durch. Erstellen Sie dazu
jeweils eine $m\times n$-Matrix, welche in jeder Zeile (=für jeden Review) jeweils eine 1 an der Index-Stelle enthält, an der ein Wort in dem entsprechenden Review vorkommt.

In [ ]:
import numpy as np
def one_hot_encode(data,n=1000):
    results = np.zeros((len(data),1000))
    for i,rev in enumerate(data): 
        results[i,rev] = 1
    return results

In [ ]:
X_train = one_hot_encode(train_data)
X_test  = one_hot_encode(test_data)
y_train = np.asarray(train_labels)
y_test = np.asarray(test_labels)

In [ ]:
X_train.shape

- Splitten Sie vom Testset die ersten 10.000 Daten ab für die Validierung während des Trainings.

In [ ]:
X_val = X_test[:10000,:]
y_val = y_test[:10000]
X_test = X_test[10000:,:]
y_test = y_test[10000:]

- Erstellen Sie ein neuronales Netz mit zwei Zwischenschichten der Größe $h_1=h_2=16$ und verwenden Sie beim Ausgang die Aktivierung sigmoid.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

n = 1000
h1 = 16
h2 = 16

model = Sequential()
model.add(Dense(h1, activation='relu', input_dim=n))
model.add(Dense(h2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.add(Dense(2, activation='softmax'))
model.summary()

- Compilieren Sie das Model mit den Optionen 
    -  loss='binary_crossentropy'
    -  metrics=['accuracy']
    
und trainieren Sie anschliessend Ihr Model mit den oben festgelegten Traings- und Validierungsdaten.

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=4, batch_size=5,validation_data=(X_val,y_val))

- Plotten Sie die Accuracy von Training und Test Set über die Epochen und berechnen Sie die accuracy bezüglich des Test-Sets.

In [ ]:
print(model.metrics_names)

In [ ]:
model.evaluate(X_val,y_val)

In [ ]:
np.mean(model.predict_classes(X_val)==y_val.reshape(-1,1))

## MNIST-Fashion Data Set (Multiclass Classification)

Das MNIST-Fashion DataSet enthält 70.000 Bilder (60.000 Training, 10.000 Test) mit den 10 Kategorien:

| Index | Item  |
|---|-------------|
|0	| T-shirt/top |
|1	| Trouser     |
|2	| Pullover    |
|3	|Dress        |
|4  |Coat         |
|5	|Sandal       |
|6	|Shirt        |
|7	|Sneaker      |
|8	|Bag          |
|9	|Ankle boot   |

- Lesen Sie Trainings- und Testdatein ein.

In [ ]:
from keras.datasets import fashion_mnist
#from keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
#(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

- Zeigen Sie ein beliebiges Bild an.

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_train[2000],cmap='gray')
plt.show()

- Führen Sie analog zur Vorlesung ein Preprocessing der Daten durch, in dem Sie
    - Für Trainings- und Testdaten die letzten beiden Dimensionen (der Größe $28\times 28$) das Datentensors auf
    eine Dimension (der Größe $28^2$) kontrahieren
    - Die Pixelwerte von $[0,255]$ auf das Intervall $[0,1]$ abbilden
    - Von den Trainingsdaten die ersten 5.000 für die Validierung absplitten

In [ ]:
import numpy as np

X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)

X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_test = y_test.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

Führen Sie dann folgende Schritte aus

- Erstellen Sie ein geeignetes Neuronales Netz für diese Multiclass Classification
- Compilieren und trainieren Sie ihr Netz
- Plotten Sie die Accuracy des Training- und Validierungssets über die Epochen
- Berechnen Sie die Accuracy bezüglich des Testsets
- Lassen Sie die Confusion-Matrix für das Testset ausgeben (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

n = 28*28
h1 = 100
h2 = 100
K = 10

model = Sequential()
model.add(Dense(h1, activation='relu', input_dim=n))
model.add(Dense(h2, activation='relu'))
model.add(Dense(K, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=8, batch_size=50,validation_data=(X_valid,y_valid))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['sparse_categorical_accuracy'],'o')
plt.plot(history.history['val_sparse_categorical_accuracy'],'o')

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(model.predict_classes(X_test),y_test)

## Boston Housing Data Set (Regression)
Das Boston Housing Data Set gibt die Preise von Häusern in Boston als Funktion der 13 Features, die in der
folgenden Tabelle gezeigt sind an.

|feature | label | content|
|--|--|--|
|0  |CRI     |     per capita crime rate by town     |
|1  |ZN      |    proportion of residential land zoned for lots over 25,000 sq.ft.|
|2  |INDUS   |    proportion of non-retail business acres per town|
|3  |CHAS    |    Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)|
|4  |NOX     |    nitric oxides concentration (parts per 10 million)|
|5  |RM      |    average number of rooms per dwelling|
|6  |AGE     |    proportion of owner-occupied units built prior to 1940|
|7  |DIS     |    weighted distances to five Boston employment centres|
|8  |RAD     |    index of accessibility to radial highways|
|9  |TAX     |    full-value property-tax rate per \$10,000|
|10 |PTRATIO |  pupil-teacher ratio by town|
|11 |B       | $1000(B_k - 0.63)^2$ where $B_k$ is the proportion of blacks by town|
|12 |MEDV    | Median value of owner-occupied homes in \$1000's|

- Lesen Sie die Daten ein.

In [1]:
from keras.datasets import boston_housing

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

Using TensorFlow backend.


In [ ]:
X_train[0]

- Bereinigen Sie Ihre Daten vom Mittelwert und skalieren Sie auf eine Varianz von 1. Berechnen dazu den den Mittelwert $\vec{\mu}$ und die Standardabweichung $\vec{\sigma}$ des Training Sets (für alle features über alles Trainingsbeispiele). Ziehen dann $\mu$ von Training und Test Set ab und teilen Sie anschliessend durch $\vec{\sigma}$.

In [ ]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train -= mean
X_train /=std

X_test -= mean
X_test /=std

- Generieren Sie ein neuronales Netz mit zwei Hiddenlayern der Größe $h_1=h_2=64$. Der Ausgabelayer erhält hier
keine Aktivierung, das es sich um eine Regression handelt.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

n = 13
h1 = 64
h2 = 64

model = Sequential()
#model.add(Dense(1, input_dim=n))
model.add(Dense(h1, activation='relu', input_dim=n))
model.add(Dense(h2, activation='relu'))
model.add(Dense(1))
model.summary()

- Kompilieren Sie das Model mit loss='mse' und  metrics=['mae'] und führen Sie anschliessend das Training
mit allen Trainingsdaten über 80 Epochen mit einem batch_size von 16 durch.

In [ ]:
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae'])

In [ ]:
model.fit(X_train,y_train,epochs=200,batch_size=16)

- Evaluieren Sie den MAE für das Test Set.

In [ ]:
#print(model.predict(X_test))
print(model.evaluate(X_train,y_train))